ModuleNotFoundError: No module named 'datasets'

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [5]:
from datasets import load_dataset
data=load_dataset('recastai/coyo-75k-augmented-captions')
data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/73480 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/935 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['url', 'key', 'image_phash', 'original_caption', 'tag', 'attribute', 'short_caption', 'detailed_caption', 'llm_caption'],
        num_rows: 73480
    })
    test: Dataset({
        features: ['url', 'key', 'image_phash', 'original_caption', 'tag', 'attribute', 'short_caption', 'detailed_caption', 'llm_caption'],
        num_rows: 935
    })
})

In [6]:
import pandas as pd
df1 = pd.DataFrame(data['train'])
df2 = pd.DataFrame(data['test'])

In [7]:
df1.head()
df1.drop(columns=['url','key','image_phash','original_caption','tag','attribute','detailed_caption','llm_caption'],inplace=True)
df1

,short_caption
0,[a table sawing machine on a white background]
1,[a brown leather recliner chair on a white bac...
2,[a blue hiking shoe with black laces]
3,[a black and white photo of a dog team pulling...
4,[a pair of key chains with a police officer on...
...,...
73475,[a chart with the words break - even analysis ...
73476,[a woman laying on the ground with her legs up]
73477,[a silver sports car parked in a garage]
73478,[a man laying on the ground with a rugby ball]


In [8]:
df1_resampled = df1.sample(frac=0.02, random_state=42)
df1_resampled.head()

,short_caption
12802,[a golf course overlooking the ocean]
55492,[a pair of earrings with saturn on them]
31413,[a row of lettuce growing in a greenhouse]
7255,[a brown and red wetsuit with a logo on the ch...
25770,[a large herd of sheep in a fenced in area]


In [9]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [10]:
from sentence_transformers import SentenceTransformer
# Extract the text data from the 'prompt' column
texts = [caption[0] for caption in df1_resampled['short_caption']]

# Load the pre-trained Sentence Transformer model
model = SentenceTransformer("all-mpnet-base-v2")

# Generate embeddings for the texts
embeddings = model.encode(texts)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
embeddings

array([[-3.30504626e-02,  3.09264893e-03, -2.58544902e-03, ...,
        -4.62786816e-02, -1.05852634e-02,  1.27637619e-02],
       [ 6.69126883e-02, -1.32738054e-02,  4.62563010e-03, ...,
         2.85434145e-02,  1.10712312e-02, -1.70887485e-02],
       [-9.33341868e-03,  2.21292488e-02, -3.10984403e-02, ...,
        -2.64592301e-02, -5.59592736e-05, -4.78781350e-02],
       ...,
       [ 1.39201069e-02, -7.38974139e-02,  1.06535358e-02, ...,
         1.88677311e-02,  3.75635438e-02, -7.35892802e-02],
       [ 1.01120761e-02,  2.38696026e-04,  1.07842479e-02, ...,
        -7.20340945e-03, -2.65346784e-02, -1.84066072e-02],
       [ 1.93740427e-02,  5.45014255e-03, -1.08352127e-02, ...,
         2.41789483e-02,  1.21098645e-02, -4.58747149e-02]], dtype=float32)

In [12]:
df_embeddings = pd.DataFrame(embeddings)
df_embeddings['short_caption'] = texts

# Save the DataFrame to a CSV file
df_embeddings.to_csv('short_caption_embeddings.csv', index=False)

# Display the first few rows of the DataFrame
print(df_embeddings.head())

          0         1         2         3         4         5         6  \
0 -0.033050  0.003093 -0.002585  0.023372 -0.025069 -0.019173  0.004039   
1  0.066913 -0.013274  0.004626  0.014954 -0.034129  0.038473 -0.028729   
2 -0.009333  0.022129 -0.031098  0.067719 -0.033459  0.016668  0.094018   
3  0.021401 -0.066085  0.000850 -0.010120  0.019578  0.010993  0.010652   
4 -0.053834 -0.010679 -0.005219  0.004814 -0.026120 -0.010065  0.081848   

          7         8         9  ...       759       760       761       762  \
0 -0.012458 -0.022461  0.010763  ... -0.025818 -0.009939 -0.007868  0.014794   
1  0.046984  0.001615  0.021564  ... -0.016344  0.033202 -0.049992  0.013940   
2  0.023549  0.083554 -0.005879  ...  0.002846  0.020323 -0.021777 -0.006365   
3  0.032798  0.009160 -0.027020  ... -0.000564  0.036705  0.047665  0.029363   
4 -0.000287 -0.004990 -0.006764  ... -0.035391  0.066639 -0.022857  0.010329   

        763       764       765       766       767  \
0 -0.008872 -